In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [5]:
def my_neo4j_wipe_out_mst_relationships():
    "wipe out mst relationships"
    
    query = "match (node)-[relationship:MST]->() delete relationship"
    session.run(query)

### Shortest Path: From Distribution Center to Drone Delivery Stations

In [12]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [13]:
#Create list of stations where drones will be placed
stations = ['Powell Street', 'North Berkeley', 'Pleasant Hill',
           'South San Francisco', 'Bay Fair', 'Downtown Berkeley',
           '16th Street Mission', 'Balboa Park', '12th Street',
           '24th Street Mission', 'Lake Merritt']

#Run shortest path from West Oakland (distribution center) to all stations in list
for station in stations:
    my_neo4j_shortest_path('depart West Oakland', 'arrive ' + station)


--------------------------------
   Total Cost:  600
   Minutes:  10.0
--------------------------------
depart West Oakland, 0, 0
red West Oakland, 0, 0
red Embarcadero, 420, 420
red Montgomery Street, 60, 480
red Powell Street, 120, 600
arrive Powell Street, 0, 600

--------------------------------
   Total Cost:  1140
   Minutes:  19.0
--------------------------------
depart West Oakland, 0, 0
red West Oakland, 0, 0
red 12th Street, 300, 300
red 19th Street, 120, 420
red MacArthur, 180, 600
red Ashby, 240, 840
red Downtown Berkeley, 180, 1020
red North Berkeley, 120, 1140
arrive North Berkeley, 0, 1140

--------------------------------
   Total Cost:  1860
   Minutes:  31.0
--------------------------------
depart West Oakland, 0, 0
yellow West Oakland, 0, 0
yellow 12th Street, 300, 300
yellow 19th Street, 120, 420
yellow MacArthur, 180, 600
yellow Rockridge, 240, 840
yellow Orinda, 300, 1140
yellow Lafayette, 300, 1440
yellow Walnut Creek, 300, 1740
yellow Pleasant Hill, 120, 1860
a